In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import collections
import category_encoders as ce

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Загружаем данные

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования
DATA_DIR = '/kaggle/input/sf-booking/'
# DATA_DIR = 'data/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'/hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.head(2)

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# Изучаем данные

In [ ]:
data.info()

Данный датасет состоит  из 17 признаков(столбцов) + 1 добавили(sample).\
Данные следующих типов представлены: 4 признака - float, 6 - int, 8 - object.\
review_date представлены типом object, данный признак можно представить datetime.\
В признаках lat и lng есть пропуски

In [ ]:
# Пропущенные данные
print(data.isnull().sum()[data.isnull().sum() > 0]) 

In [ ]:
# Дубликаты 
print('Количество дубликатов: {}'.format(data[data.duplicated()].shape[0]))

In [ ]:
data[data.duplicated()].head()

При быстром просмотре на дублирующие данные, можно предположить что они таковыми не считаются.\
Для подверждение, выведим количество уникальных данных для каждого признака

In [ ]:
data[data.duplicated()].nunique()

# Очистка данных, создание новых признаков  

## 1.Hotel_address

In [ ]:
print(data['hotel_address'][2] + '\n')
print(data['hotel_address'][10] + '\n')
print(data['hotel_address'][278] + '\n')
print(data['hotel_address'][1258] + '\n')
print(data['hotel_address'][515734])

Из признака 'hotel_address' можно извлечь информацию о стране и городе отеля. Извлекем город и страну и подчистим адрес от дублирующих данных(город и страна)  

In [ ]:
# Функция извлечения города в котором находится отель
def city_hotels(address):
    ls_city = address.split()
    if ls_city[-1] == 'Kingdom':
        return ls_city[-5]
    return ls_city[-2]

In [ ]:
# Функция извлечения страны в котором находится отель
def country_hotels(address):
    ls_city = address.split()
    if ls_city[-1] == 'Kingdom':
        return ' '.join(ls_city[-2:])
    return  ls_city[-1]

In [ ]:
# Функция для очистки данных  от названия города и страну
def address_hotels(address):
    ls_city = address.split()
    if ls_city[-1] == 'Kingdom':
        return ' '.join(ls_city[:-5])
    return ' '.join(ls_city[:-2])

In [ ]:
# преобразуем признак 'hotel_address' и создаем два новых 'country_hotel' и 'city_hotel'
data['city_hotel'] = data['hotel_address'].apply(city_hotels)
data['country_hotel'] = data['hotel_address'].apply(country_hotels)
data['hotel_address'] = data['hotel_address'].apply(address_hotels)
data.info()

In [ ]:
print(data['city_hotel'].unique())
print(data['country_hotel'].unique())

In [ ]:
data['city_hotel'].value_counts()

In [ ]:
ax = sns.histplot(data['city_hotel'])
ax.set(xlabel='Город', ylabel='Количество рецензентов')
ax.set_title('Распределение количества рецензентов для отелей взависимости от города')

Самый популярный город - Лондон, т.к. больше всего оставили оценки для отелей Лондона.  

## 2. Review_date

In [ ]:
# Преобразуем в тип datetime
data['review_date'] = pd.to_datetime(data['review_date'], dayfirst=True)

#Создадим новый признак квартал в котором была выставлена оценка и написан отзыв
data['review_quarter'] = data['review_date'].dt.to_period('Q')
data['review_quarter'][:5]

In [ ]:
# Выведим в признак review_quarter только номер квартал и преобразуем в тип int
data['review_quarter'] = data['review_quarter'].apply(lambda x: int(str(x)[-1]))

In [ ]:
data['review_quarter'].value_counts()

Больше отзывов приходиться на 2 и 3 кварталы, это апрель - сентябрь. На эти месяцы приходиться сезон отпусков.

In [ ]:
#удалим столбец review_date
data.drop('review_date', axis=1, inplace=True)

## 3. lat и lnf. Заполняем пропуски

In [ ]:
# В пропущенных значений широты и  долготы,  заполняем средним значением 
# координат отелей, находящиеся в этом же городе
for city in data['city_hotel'].value_counts().index:
    mask = data['city_hotel'] == city
    if data[mask].isnull().sum()['lat'] > 0 and data[mask].isnull().sum()['lng'] > 0:
        value = {'lat': data[mask]['lat'].mean(), 'lng': data[mask]['lng'].mean()}
        data[mask] = data[mask].fillna(value)

In [ ]:
print(data.isnull().sum()[data.isnull().sum() > 0])

## 4. Negative_review и positive_review

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
sent_analyzer = SentimentIntensityAnalyzer()

In [ ]:
data['negative_review'][4511]

In [ ]:
sent_analyzer.polarity_scores(data['negative_review'][4511])

In [ ]:
data['positive_review'][4511]

In [ ]:
sent_analyzer.polarity_scores(data['positive_review'][4511])

In [ ]:
data['negativ_neg'] = data['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['neg'])
data['negativ_neu'] = data['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['neu'])
data['negativ_pos'] = data['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['pos'])
data['negativ_compound'] = data['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['compound'])

In [ ]:
data['positive_neg'] = data['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['neg'])
data['positive_neu'] = data['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['neu'])
data['positive_pos'] = data['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['pos'])
data['positive_compound'] = data['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['compound'])

## 5. Days_since_review

In [ ]:
data['days_since_review'][:5]

In [ ]:
# Преобразуем данный признак в тип int.
data['days_since_review'] = data['days_since_review'].apply(lambda x: int(x.split()[0]) )

## 6. Reviewer_nationality

In [ ]:
data['reviewer_nationality'][0]

In [ ]:
data['reviewer_nationality'][20]

В признаке "reviewer_nationality" в данных есть пробел  в начале и конце  строки. Уберем его.
Сделаем принак, который показывает являестся ли reviewer гражданином страны, где он оставлял  отзыв отеля

In [ ]:
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: ' '.join(x.split()))
data['citizenship'] = np.where(data['country_hotel'] == data['reviewer_nationality'], 1, 0)

## 7. Tag 

In [ ]:
data['tags'][0]

In [ ]:
data['tags'][358]

In [ ]:
def transformation_tag(tag):
    ls = tag.split(" ', ' ")
    ls[0] = ls[0][3:]
    ls[-1] = ls[-1][:-3]
    return ls

data['tags'] = data['tags'].apply(transformation_tag)

In [ ]:
data['tags'][0]

In [ ]:
data['tags'][0][0]

Данные представлены ввиде списка

In [ ]:
# Выведим самые популярные теги (50)
c = collections.Counter()
for tags in data['tags']:
    for tag in tags:
        c[tag] += 1

c.most_common()[0:50:]

По часто встречающимся можно создать новые признаки

In [ ]:
# количество ночей проведенных в отеле
def numerical(string):
    for elem in string.split():
        if elem.isdigit():
            return int(elem)
    return False

def total_night(tags):
    for tag in  tags:
        if 'night' in tag and numerical(tag) != False:
            return numerical(tag)
    return 0

data['total_nights'] = data['tags'].apply(total_night)

In [ ]:
data['total_nights'].value_counts()

Определим является ли у reviewer туристическая поездка или бизнес поездка

In [ ]:
def leisure_trip(tags):
    for tag in tags:
        if ('Leisure trip' in tag) or ('travel' in tag):
            return 1
    return 0 

def business_trip(tags):
    for tag in tags:
        if ('Business trip' in tag):
            return 1
    return 0


In [ ]:
data['leisure_trip'] = data['tags'].apply(leisure_trip)
data['business_trip'] = data['tags'].apply(business_trip)

Определим признаки в отели номер снимали пара или семья с детьми или групповая поездка

In [ ]:
def couple(tags):
    for tag in tags:
        if ('Couple' in tag):
            return 1
    return 0


def children(tags):
    for tag in tags:
        if ('children' in tag):
            return 1
    return 0

def group(tags):
    for tag in tags:
        if ('Group' in tag):
            return 1
    return 0

In [ ]:
data['couple'] = data['tags'].apply(couple)
data['children'] = data['tags'].apply(children)
data['group'] = data['tags'].apply(group)

Выведим признак c какого устройства отправлен тэг

In [ ]:
def submitted_mobile(tags): 
    for tag in tags:
        if 'Submitted from a mobile device' in tag:
            return 1
    return 0


In [ ]:
data['submitted_mobile'] = data['tags'].apply(submitted_mobile)

Отдельно выделим признаки описывающие роскошные номера

In [ ]:
def luxe(tags):
    for tag in tags:
        if 'Deluxe' in tag:
            return 1
        elif 'Luxury' in tag:
            return 1
    return 0

In [ ]:
data['luxe'] = data['tags'].apply(luxe)

## 8. Hotel_name

Отели представленные в данных могут пренадлежать международным брендам гостинечного бизнеса
Возьмем ТОП-10 гостиничных брендов в стоимостном выражении: 
* Hilton
* Marriott
* Hyatt
* Sheraton
* Holiday
* Courtyard
* Hampton
* Mercure
* Ramada
* SHANGRI-LA ASIA

Определим новый признак пренадлежности отеля к определенному бренду

In [ ]:
brand = ['hilton', 'marriott', 'hyatt', 'sheraton', 'holiday', 'courtyard', 'hampton', 'mercure', 'ramada', 'shangri-la']

In [ ]:
def brand(name_hotels):
    brand = ['hilton', 'marriott', 'hyatt', 'sheraton', 'holiday', 'courtyard', 'hampton', 'mercure', 'ramada', 'shangri-la']
    for elem in name_hotels.split():
        if elem.lower() in brand:
            return elem
    return 'other'

In [ ]:
# создадим новый признак hotel_brand
data['brand'] = data['hotel_name'].apply(brand)

In [ ]:
data['brand'].value_counts()

# Преобразование признаков

Выполним кодирование номинального категориального признака city_hotel методом OneHot Encoding

In [ ]:
encoder = ce.OneHotEncoder(cols=['city_hotel', 'brand']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data[['city_hotel','brand']])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
data.info()

In [ ]:
plt.rcParams['figure.figsize'] = (50,50)
sns.heatmap(data.drop(['sample'], axis=1).corr(), fmt='.2g', annot=True, cmap= 'coolwarm', 
            linewidths=2, linecolor='black')

# Обучение модели

In [ ]:
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)